<a href="https://colab.research.google.com/github/mayaraperroni9/datascience/blob/main/Classificao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para esse exercício, vamos utilizar o dataset Iris. Ele descreve atributos sobre 3 tipos de flores.
O objetivo é classificar qual o tipo de flor de acordo com os atributos disponíveis. Vamos trabalhar apenas com as duas primeiras classes para que
o problema de classificação binária.

In [ ]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import numpy as np

iris = load_iris()
# Nessa primeira parte, vamos trabalhar apenas com as duas primeiras features e as duas primeiras classes
X = iris.data[:100, :4]
y = iris.target[:100]


---

### Questão 1.


- a) Utilizando o sklearn, defina uma [MLP para classificação binária](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) com a seguinte configuração:
    - função de ativação ReLU;
    - duas camadas escondidads com 10 neurônios cada;
    - taxa de aprendizado igual a 1e-2;
    - utilizando o algoritmo de otimização de gradiente descendente estocástico;
    - utilizando 10 iterações máximas (épocas);
    - use random_state=1234


- b) Treine a MLP definida no conjunto Iris simplificado definido na questão anterior, e calcule a cross-entropy loss binária seguindo a definição a seguir.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234) #precisa ser o mesmo que o do mlp??

# Definir o classificador MLP
mlp = MLPClassifier(hidden_layer_sizes=(10, 10),
                    activation='relu',
                    learning_rate_init=0.01,
                    solver='sgd',
                    max_iter=10,
                    random_state=1234)

# Treinar o MLP
mlp.fit(X_train, y_train)

# Prever probabilidades no conjunto de teste
y_pred = mlp.predict_proba(X_test)[:, 1]

# Calcular a cross-entropy loss binária
loss = log_loss(y_test, y_pred)

print("Cross-Entropy Loss:", loss)


Cross-Entropy Loss: 0.5289903709432386


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


---

### Questão 2.

Para avaliar os modelos que serão testados, implemente a função `evaluate_model()`. Essa função recebe um modelo de classificador genérico (`model`) e avalia sua acurácia utilizando **10-fold stratified cross-validation**, retornando a média das acurácias de cada fold. O parâmetro `X` indica os dados e `y` os labels.
- Sugestão: há duas formas de implementar a validação cruzada: treinar manualmente os modelos nos [splits gerados](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) ou utilizar a função [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) do sklearn. Atente-se ao cálculo da acurácia.

- Para garantir uma melhor performance dos algoritmos, faça o preprocessamento desses dados através da classe `sklearn.preprocessing.StandardScaler`.

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

def evaluate_model(model, X, y):
    # Criar um pipeline que primeiro normaliza os dados e depois aplica o modelo
    pipeline = make_pipeline(StandardScaler(), model)    #ACHO QUE TEM QUE PRÉ-PROCESSAR OS DADOS ANTES DE APLICAR O MODELO, NÃO?

    # Configurar a validação cruzada estratificada
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

    # Calcular as acurácias de cada fold utilizando o pipeline
    accuracies = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

    # Retornar a média das acurácias
    return accuracies.mean()

# Exemplo de como usar a função
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier

# Definir um modelo, por exemplo, MLP
model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=1234)

# Avaliar o modelo
mean_accuracy = evaluate_model(model, X_train, y_train) #melhor usar o train não?
print(f"A média da acurácia com 10-fold stratified cross-validation é: {mean_accuracy:.4f}")


A média da acurácia com 10-fold stratified cross-validation é: 1.0000


---

### Questão 3.

Agora para estruturar e organizar melhor nossos testes, vamos utilizar as estruturas de dicionário do Python. Por exemplo, se formos definir dois modelos de Multi-Layer Perceptron, podemos escrever:

```
experimentos = {
    "MLP camada escondida (5,)": MLPClassifier(hidden_layer_sizes=(5,),
    "MLP camada escondida (5,5)": MLPClassifier(hidden_layer_sizes=(5,5)        
}
```

Isso pode ser feito pois o Python trata funções como funções de primeira classe. Isso é, funções podem ser tratadas como variáveis.

Portanto, defina um dicionário de experimentos com ao menos 3 modelos de MLP (`sklearn.neural_network.MLPClassifier`). Para isso varie parâmetros como o número de camadas escondidas, função de ativação e número de neurônios.

- Dica: Ver documentação em https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
- Utilize um número de iterações >= 50 para garantir convergência.
- Experimente diferentes taxas de aprendizado e número máximo de iterações (épocas) de forma a garantir convergência no treino.

In [ ]:
from sklearn.neural_network import MLPClassifier

experimentos = {
    "MLP 1 camada (10,)": MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, tol=1e-4, learning_rate_init=0.001, solver='sgd', momentum=0.9, n_iter_no_change=20, alpha=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    "MLP 2 camadas (10, 10)": MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000, tol=1e-4, learning_rate_init=0.001, solver='sgd', momentum=0.9, n_iter_no_change=20, alpha=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    "MLP 3 camadas (10, 10, 10)": MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000, tol=1e-4, learning_rate_init=0.001, solver='sgd', momentum=0.9, n_iter_no_change=20, alpha=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
}



---

### Questão 4.

- a) Para cada modelo instanciado na Questão 3, utilize a função criada na questão 2 para calcular sua acurácia. Exiba o nome do modelo e sua acurácia.
- b) Determine qual o melhor classificador dentre os especificados e justifique sua escolha.

In [ ]:
# Avaliar cada modelo
for nome, modelo in experimentos.items():
    accuracy = evaluate_model(modelo, X, y)
    print(f"{nome}: Acurácia = {accuracy}")

# Identificar o melhor modelo (exemplo de comparação simples)
best_model = max(experimentos, key=lambda x: evaluate_model(experimentos[x], X, y))
print(f"Melhor modelo: {best_model}")


MLP 1 camada (10,): Acurácia = 1.0
MLP 2 camadas (10, 10): Acurácia = 1.0


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP 3 camadas (10, 10, 10): Acurácia = 1.0
Melhor modelo: MLP 1 camada (10,)
